In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [23]:
from keras.applications import MobileNet

# MobileNetV2 takes input of 224x224 images
img_rows = 224
img_cols = 224

mn_model = MobileNet(weights='imagenet',
                       include_top=False,
                       input_shape=(img_rows, img_cols, 3))

# Freezing the bottom_model layers to preserve original weights.
for layer in mn_model.layers:
    layer.trainable=False

In [46]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization

# Creating top_model aka Fully Connected layers
def addTop(bottom_model, num_classes, D=4096):
    top_model = bottom_model.output
    top_model = Flatten(name='Flatten')(top_model)
    top_model = Dense(D,activation='relu')(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(D, activation='relu')(top_model)
    top_model = Dropout(0.5)(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model

In [47]:
num_classes=5
FC_head = addTop(mn_model, num_classes)
model = Model(inputs=mn_model.input, outputs=FC_head)
print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [48]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/My Drive/Datascience/face_recognition/train'
val_dir = '/content/drive/My Drive/Datascience/face_recognition/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    vertical_flip=False,
    width_shift_range=0.4,
    height_shift_range=0.4,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_rows, img_cols),
    class_mode='categorical',
    batch_size=8,
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_rows, img_cols),
    class_mode='categorical',
    batch_size=8,
    shuffle=False
)

Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [49]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint('face_recog.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1
                            )

earlystopping = EarlyStopping(monitor='val_loss',
                             patience=3,
                             min_delta=0,
                             verbose=1)

callbacks = [checkpoint, earlystopping]

In [50]:
from keras.optimizers import RMSprop
optimizer = RMSprop(lr=1e-5, decay=1e-7)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [51]:
history = model.fit_generator(train_generator, 
                             epochs=10,
                             validation_data=val_generator,
                             callbacks=callbacks)

Epoch 1/10
12/12 [==============================] - 3s 247ms/step - loss: 4.0472 - accuracy: 0.1935 - val_loss: 0.4543 - val_accuracy: 0.3200

Epoch 00001: val_loss improved from inf to 0.45426, saving model to face_recog.h5
Epoch 2/10
12/12 [==============================] - 1s 70ms/step - loss: 3.4341 - accuracy: 0.2796 - val_loss: 1.0556 - val_accuracy: 0.4400

Epoch 00002: val_loss did not improve from 0.45426
Epoch 3/10
12/12 [==============================] - 2s 147ms/step - loss: 3.0905 - accuracy: 0.3548 - val_loss: 0.5015 - val_accuracy: 0.4000

Epoch 00003: val_loss did not improve from 0.45426
Epoch 4/10
12/12 [==============================] - 2s 140ms/step - loss: 3.0903 - accuracy: 0.3441 - val_loss: 0.4317 - val_accuracy: 0.5200

Epoch 00004: val_loss improved from 0.45426 to 0.43168, saving model to face_recog.h5
Epoch 5/10
12/12 [==============================] - 1s 87ms/step - loss: 3.1108 - accuracy: 0.2796 - val_loss: 0.1644 - val_accuracy: 0.4800

Epoch 00005: val_

In [52]:
import cv2
import numpy as np

In [53]:
print(train_generator.class_indices)

{'ben_afflek': 0, 'elton_john': 1, 'jerry_seinfeld': 2, 'madonna': 3, 'mindy_kaling': 4}


In [54]:
path = '/content/drive/My Drive/Datascience/face_recognition/test'
faces_dict = {'[0]':'Ben Afflek', '[1]':'Elton John', '[2]':'Jerry Seinfeld', '[3]':'[Madonna]', '[4]':'Mindy Kaling'}
font = cv2.FONT_HERSHEY_SIMPLEX
green_color = (0, 255, 0)

def diplay_photo(path):
    photo = cv2.imread(path)
    ori_photo = photo.copy()
    ori_photo = cv2.resize(ori_photo, (800,600))
    
    photo = cv2.resize(photo, (224, 224), interpolation=cv2.INTER_LINEAR)
    photo = photo/255.
    photo = photo.reshape(1, 224, 224, 3)
    res = np.argmax(model.predict(photo, 1, verbose = 0), axis=1)
    print(res)
    name=faces_dict[str(res)]
    print(name)
    cv2.putText(ori_photo, name, (10, 11), font, 1, green_color, 2)
    # while(True):
    #     cv2.imshow("ori_photo", name)
    #     if cv2.waitKey()==27: # Enter Key
    #         cv2.destroyAllWindows()
    #         break

In [56]:
import os
path = '/content/drive/My Drive/Datascience/face_recognition/test/'
# diplay_photo(path)
for image_path in os.listdir(path):
    print(path+image_path)
    diplay_photo(path+image_path)


/content/drive/My Drive/Datascience/face_recognition/test/madonna_sc_768x1024.png
[3]
[Madonna]
/content/drive/My Drive/Datascience/face_recognition/test/283920_original.jpg
[2]
Jerry Seinfeld
/content/drive/My Drive/Datascience/face_recognition/test/Mindy-Kaling.jpg
[4]
Mindy Kaling
